<a href="https://colab.research.google.com/github/Sadiagul914/AI201/blob/main/Project_2_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
!pip install -qU langchain-community langchain-pinecone langchain-google-genai -qU pypdf --quiet --upgrade langchain-text-splitters

In [96]:
!pip install pinecone-client
!pip install langchain_pinecone
!pip install PineconeVectorestore

ERROR: Could not find a version that satisfies the requirement PineconeVectorestore (from versions: none)
ERROR: No matching distribution found for PineconeVectorestore


In [97]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

#Creating pinecone vector db

In [108]:
from google.colab import userdata
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone ,ServerlessSpec
vector_store= PineconeVectorStore(index_name= index_name, embedding= embeddings)
pinecone_api_key= userdata.get('PINECONE_API_KEY')
pc=Pinecone(api_key=pinecone_api_key)

#Creating Index

In [126]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

index_name = "quickstart4"
pc = Pinecone(api_key='pcsk_5d9zDY_QsD3jJd9JsPH53u2r9gr5HjaHja3x7G6iDfeK26fy6q8zwUYEKr2pxrZ1KiwJz9')
pc.delete_index(index_name)
pc.create_index(
      name=index_name,
      dimension=768, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)


#Google Embeddings


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings= GoogleGenerativeAIEmbeddings(model='models/embedding-001')
embeddings.embed_query("hello world")


[0.04909781739115715,
 -0.044328317046165466,
 -0.025365281850099564,
 -0.030721040442585945,
 0.019068587571382523,
 -0.010923865251243114,
 0.03320307657122612,
 -0.009435197338461876,
 0.014225783757865429,
 0.011143019422888756,
 0.038166288286447525,
 0.05900086462497711,
 -0.01907157711684704,
 -0.07919705659151077,
 0.008873151615262032,
 -0.01893804408609867,
 0.013136426918208599,
 -0.010632799006998539,
 0.01072288304567337,
 -0.009800860658288002,
 -0.005997073836624622,
 0.0035213204100728035,
 -0.0425376296043396,
 -0.017927661538124084,
 0.00820374395698309,
 0.021045953035354614,
 -0.0070729125291109085,
 -0.06959639489650726,
 0.003866896266117692,
 0.06563343107700348,
 -0.03162197768688202,
 0.016947802156209946,
 -0.0678708553314209,
 0.0121560487896204,
 0.03733306750655174,
 -0.043934062123298645,
 0.024457911029458046,
 0.025291720405220985,
 -0.007016133517026901,
 -0.0070653194561600685,
 0.020251881331205368,
 -0.09896384924650192,
 -0.025554955005645752,
 -0.0

#Document Loader

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader


textloader= TextLoader("documents.txt")
documents= textloader.load()


#Text Spiltter


In [138]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter= CharacterTextSplitter(chunk_size=50, chunk_overlap=0)
docs= text_splitter.split_documents(documents)
print(docs)

[Document(metadata={'source': 'documents.txt', 'text': '\nVideo provides a powerful way to help you prove your point. When you click Online Video, you can paste in the embed code for the video you want to add. You can also type a keyword to search online for the video that best fits your document. To make your document look professionally produced, Word provides header, footer, cover page, and text box designs that complement each other. For example, you can add a matching cover page, header, and sidebar. Click Insert and then choose the elements you want from the different galleries. Themes and styles also help keep your document coordinated. When you click Design and choose a new Theme, the pictures, charts, and SmartArt graphics change to match your new theme. When you apply styles, your headings change to match the new theme. Save time in Word with new buttons that show up where you need them.\nTo change the way a picture fits in your document, click it and a button for layout opti

#uploading to pinecone

In [140]:
!pip install tqdm

In [153]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
vectors=[]
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)
print("Document content:", doc.page_content) # Print the document content
print("Vector:", vector) # Print the vector generated for debugging

    # Generate a unique ID for each document (e.g., UUID or hash)
doc_id = str(hash(doc.page_content))
metadata = {"text": "Click Insert and then choose the elements you want from the different galleri"} # Add the text content to the metadata
    # Upsert the vector with a unique ID and metadata
index.upsert(vectors=[(doc_id, vector, metadata)]) #Include metadata during upsert



100%|██████████| 1/1 [00:00<00:00,  5.22it/s]


Document content: Video provides a powerful way to help you prove your point. When you click Online Video, you can paste in the embed code for the video you want to add. You can also type a keyword to search online for the video that best fits your document. To make your document look professionally produced, Word provides header, footer, cover page, and text box designs that complement each other. For example, you can add a matching cover page, header, and sidebar. Click Insert and then choose the elements you want from the different galleries. Themes and styles also help keep your document coordinated. When you click Design and choose a new Theme, the pictures, charts, and SmartArt graphics change to match your new theme. When you apply styles, your headings change to match the new theme. Save time in Word with new buttons that show up where you need them.
To change the way a picture fits in your document, click it and a button for layout options appears next to it. When you work on 

{'upserted_count': 1}

#Data Retrieval Using Semantic Search

In [161]:
#data Retrieval
results = vector_store.similarity_search(
    "Click Insert and then choose the elements",
    k=1,
    filter={"text": "Click Insert and then choose the elements you want from the different galleri"},
    )
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Click Insert and then choose the elements you want from the different galleri [{}]
